#### H2O AutoML

H2O는 오픈소스 머신러닝 플랫폼으로 다양한 머신러닝 모형과 딥러닝 등을 구축하여 AutoML과 같은 최신 알고리즘을 제공한다.

In [ ]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.40.0.4-py2.py3-none-any.whl size=177697886 sha256=860610bf7c99302b1e046f7070ee32304e0716a0f1cc17ab26035d2bcb03c862
  Stored in directory: /root/.cache/pip/wheels/43/f2/b0/5bb4d702a0467e82d77c45088db3eef25114c26b0eec8e7f6a
Successfully built h2o


In [ ]:
import sys, os, os.path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle #파이썬에서 텍스트 이외의 자료형을 파일로 저장하기 위해 사용하는 모듈

import h2o
from h2o.automl import H2OAutoML

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

h2o.init()으로 h20 인스턴스를 생성해야 함

h2o에서 사용할 클러스터의 가동 시간, 시간대, 버전, 버전 수명, 클러스터 이름, 할당된 하드웨어 리소스 (노드 수, 메모리, 코어 수), 연결 URL, 노출된 H2O API 확장 및 사용된 파이썬 버전 정보가 출력됨

In [ ]:
h2o.init(nthreads=-1, max_mem_size=12)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.19" 2023-04-18; OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp4ud307du
  JVM stdout: /tmp/tmp4ud307du/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp4ud307du/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_unknownUser_y87ne5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


##### 1. 데이터 준비

H2OFrame이라는 자체 데이터프레임이 존재



In [ ]:
train=train.iloc[:,4:]
test=test.iloc[:,4:]

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train, test_size=0.2, shuffle=True)

h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)
h2o_valid = h2o.H2OFrame(valid)

for i in h2o_train.columns:
  h2o_train[i] = h2o_train[i].asfactor()  #asfactor함수는 데이터 중 label이 어떤 column에 해당하는지 알려줌

for i in h2o_test.columns:
  h2o_test[i] = h2o_test[i].asfactor()

for i in h2o_valid.columns:
  h2o_valid[i] = h2o_valid[i].asfactor()

h2o_train.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:209
Cols:17

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
mins,,,,,,,,,,,,,,,,,
mean,,,,,,,,,,,,,,,,,
maxs,,,,,,,,,,,,,,,,,
sigma,,,,,,,,,,,,,,,,,
zeros,,,,,,,,,,,,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,2,G G,A G,C A,A A,C C,A A,A A,G G,G A,G G,G G,A A,A G,A A,G A,B
1,2,A G,A G,C A,A A,C A,A G,A A,G G,A A,G G,A G,G A,A G,A A,A A,B
2,2,G G,A G,C C,G A,C A,A G,A A,G G,A A,G G,A A,A A,A A,A A,G A,B


##### 2. 모델 준비

< H20AutoML() 파라미터 >

- max_runtime_secs : 최대 몇 초간 탐색을 할 것인지 정하는 파라미터. 컴피티션의 경우 이 값을 키워서 최대의 성능을 가지는 모델을 찾아내는 것이 좋을 수 있음.

- exclude_algos : 어떤 알고리즘을 제외할 것인지를 선언할 수 있음. xgboost와 stackedEnsemble을 제외한 이유는 실습용이기 때문에 시간이 너무 오래걸리는 두 모델은 일부러 제외하고 진행. 컴피티션을 진행한다면 모든 모델을 넣고 돌리셔도 무방.

In [ ]:
aml = H2OAutoML(max_runtime_secs=(6000), max_models=None,
                exclude_algos=['XGBoost', 'StackedEnsemble'],
                seed=17)

In [ ]:
train.columns

Index(['trait', 'SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06',
       'SNP_07', 'SNP_08', 'SNP_09', 'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13',
       'SNP_14', 'SNP_15', 'class'],
      dtype='object')

In [ ]:
columns_list = train.columns
columns_list = columns_list.to_list()
columns_list.remove('class')

In [ ]:
aml.train(x=columns_list, y='class',
          training_frame=h2o_train,
          validation_frame=h2o_valid)

lb = aml.leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
out_path = './save_h2omodel'

for m_id in model_ids:
  mdl = h2o.get_model(m_id)
  h2o.save_model(model=mdl, path=out_path, force=True)

h2o.export_file(lb, os.path.join(out_path, 'aml_leaderboard.h2o'), force=True)

AutoML progress: |
01:27:25.814: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

██
01:27:31.510: GBM_1_AutoML_1_20230612_12725 [GBM def_5] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_1_AutoML_1_20230612_12725.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 167.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 167.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 167.0.
ERRR on field: _min_rows: The dataset size is too small to spli

leaderboard: 모델의 성능을 보기 위해 쓰는 함수

export_file: leaderboard를 저장해 언제든 불러올 수 있음

##### 3. 모델 선정

학습이 끝나고 import_file()을 통해 export한 leaderboard를 불러올 수 있음

In [ ]:
lb = h2o.import_file('./save_h20model/aml_leaderboard.h2o')
lb.head(rows=10)

In [ ]:
performance = aml.leader.model_performance(h2o.valid)

In [ ]:
model_id = aml.leader.model_id #최고 모델명
print(model_id)

h2o.load_model()을 통해 저장해둔 경로의 best model을 load하기

In [ ]:
model_01 = h2o.load_model('')

In [ ]:
model_01

##### 4. 피처 중요도

In [ ]:
aml.leader.varimp_plot()

##### 5. 예측하기

그냥 model_01.predict를 하게되면 h2o 자체 데이터프레임으로 반환되므로 pd의 dataframe 형식으로 반환해주기 위해 as_data_frame을 사용해줌

In [ ]:
best_pred = model_01.predict(h2o_test).as_data_frame().predict

In [ ]:
best_pred

In [ ]:
submit = pd.read_csv('/content/sample_submission.csv')
submit

In [ ]:
submit['class'] = best_pred
submit.to_csv('h2o_best.csv', inplace=False)

피쳐엔지니어링 하나 없이 양호한 성능이 나오는 것으로 확인됨